In [1]:
import numpy as np

from Related-Codes.DCNAutoencoder import DCNAutoencoder, DCNCDAutoencoder
from Related-Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from Related-Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding

import torch

import os
import sys

from Datasets.Datasets_Functions import *
from Visualization import Visualization

SyntaxError: invalid syntax (3428067944.py, line 3)

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

In [ ]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)
print(hashmap)
dataset_name = "emnist_balanced_digits"
dataset_properties = hashmap[dataset_name]
batch_size = dataset_properties['batch_size'] = 256
#n_clusters = dataset_properties['n_clusters'] = 6
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)
print(input_dim)

In [ ]:
visualization = Visualization()

### IDec

In [ ]:
n_clusters = dataset_properties['n_clusters']

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size']

# Pre-Training Epochs and Learning Rate
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
n_epochs = 100
lr = 1e-3
alpha = 1.0
gamma = 0.1
momentum = 0.9
latent_dim = 10
n_channels = 1
is_MLP_AE = False

if(is_MLP_AE):
    idec_autoencoder = IDECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim)
else:  
    input_dim = 1
    idec_autoencoder = IDECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, n_channels=n_channels) 
idec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
idec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
idec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha, gamma=gamma)
idec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
idec_autoencoder.set_path()
idec_autoencoder = idec_autoencoder.to(device)

pretrain = True
if(pretrain):
    idec_autoencoder.pretrain_autoencoder()
    idec_autoencoder.save_pretrained_weights()
else:
    model_save_path = idec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    idec_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
idec_autoencoder.kmeans_initialization()
idec_autoencoder.train_autoencoder()
cluster_centers = idec_autoencoder.get_cluster_centers()
_, idec_autoencoder_reduced_data, labels = idec_autoencoder.get_latent_data()
General_Functions().save_excel(idec_autoencoder.data_dir_path, idec_autoencoder.df_eval)
#visualization.plot(idec_autoencoder_reduced_data, labels, labels, cluster_centers, idec_autoencoder.data_dir_path)
visualization.plot_tsne(idec_autoencoder_reduced_data, labels, labels, cluster_centers, idec_autoencoder.data_dir_path)

In [ ]:
idec_autoencoder.kmeans_initializatio()

### Dec

In [ ]:
n_clusters = dataset_properties['n_clusters']

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size']

# Pre-Training Epochs and Learning Rate
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
n_epochs = 100
lr = 5e-4
alpha = 1.0
momentum = 0.9
latent_dim = 10
n_channels = 1
is_MLP_AE = False

if(is_MLP_AE):
    input_dim = 784
    dec_autoencoder = DECAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim)
else:  
    input_dim = 1
    dec_autoencoder = DECCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, n_channels=n_channels) 
dec_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dec_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dec_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, momentum=momentum, alpha=alpha )
dec_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dec_autoencoder.set_path()
dec_autoencoder = dec_autoencoder.to(device)

pretrain = True
if(pretrain):
    dec_autoencoder.pretrain_autoencoder()
    dec_autoencoder.save_pretrained_weights()
else:
    model_save_path = dec_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dec_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
dec_autoencoder.kmeans_initialization()
dec_autoencoder.train_autoencoder()
cluster_centers = dec_autoencoder.get_cluster_centers()
_, dec_autoencoder_reduced_data, labels = dec_autoencoder.get_latent_data()
General_Functions().save_excel(dec_autoencoder.data_dir_path, dec_autoencoder.df_eval)
visualization.plot(dec_autoencoder_reduced_data, labels, labels, cluster_centers, dec_autoencoder.data_dir_path)
#visualization.plot_tsne(dec_autoencoder_reduced_data, labels, labels, cluster_centers, dec_autoencoder.data_dir_path)

In [ ]:
dec_autoencoder.kmeans_initializatio()

### Dcn

In [ ]:

"""
parser.add_argument('--n-classes', type=int, default=10,
                    help='output dimension')
"""
n_clusters = dataset_properties['n_clusters'] = 6

# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size'] = 256

# Pre-Training Epochs and Learning Rate
"""
parser.add_argument('--pre-epoch', type=int, default=50, 
                    help='number of pre-train epochs')
"""
n_pret_epochs = 100
pret_lr = 1e-3

# Lamdas, Training Epochs and Learning Rate
"""
parser.add_argument('--epoch', type=int, default=100,
                    help='number of epochs to train')
"""
n_epochs = 100
"""
parser.add_argument('--lr', type=float, default=1e-4,
                    help='learning rate (default: 1e-4)')
"""
lr = 1e-4
"""
parser.add_argument('--lamda', type=float, default=1,
                    help='coefficient of the reconstruction loss')
"""
lamda = 1
"""
parser.add_argument('--beta', type=float, default=1,
                    help=('coefficient of the regularization term on '
                          'clustering'))
"""
beta = 1e-5
"""
parser.add_argument('--latent_dim', type=int, default=10,
                    help='latent space dimension')
"""
latent_dim = 10
n_channels = 1
is_MLP_AE = True

if(is_MLP_AE):
    input_dim = 784
    dcn_autoencoder = DCNAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim)
else:  
    input_dim = 1
    dcn_autoencoder = DCNCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, n_channels=n_channels) 
dcn_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dcn_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dcn_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, lamda=lamda, beta=beta )
dcn_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dcn_autoencoder.set_path()
dcn_autoencoder = dcn_autoencoder.to(device)

pretrain = True
if(pretrain):
    dcn_autoencoder.pretrain_autoencoder()
    dcn_autoencoder.save_pretrained_weights()
else:
    model_save_path = dcn_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dcn_autoencoder.load_state_dict(torch.load(model_save_path))

In [ ]:
dcn_autoencoder.train_autoencoder()
cluster_centers = dcn_autoencoder.get_cluster_centers()
_, dcn_autoencoder_reduced_data, labels = dcn_autoencoder.get_latent_data()
General_Functions().save_excel(dcn_autoencoder.data_dir_path, dcn_autoencoder.df_eval)
visualization.plot_tsne(dcn_autoencoder_reduced_data, labels, labels, cluster_centers, dcn_autoencoder.data_dir_path)

### Pca

In [ ]:
# Create a PCA object
pca = PCA(n_components=2)  # Choose the number of components

# Fit and transform the data to the lower-dimensional space
pca_reduced_data = pca.fit_transform(data_np)

visualization.plot(pca_reduced_data, labels, labels, np.empty(0), "./Results/r100/PCA")

### Svd

In [ ]:
# Apply Truncated SVD for dimensionality reduction
svd = TruncatedSVD(n_components=2)
svd_reduced_data = svd.fit_transform(data_np)

visualization.plot(svd_reduced_data, labels, labels, np.empty(0), "./Results/r100/SVD")

### Nmf

In [ ]:
# Apply NMF for dimensionality reduction
nmf = NMF(n_components=2)
nmf_reduced_data = nmf.fit_transform(data_np)

visualization.plot(nmf_reduced_data, labels, labels, np.empty(0), "./Results/r100/NMF")

### Lle

In [ ]:
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)  # Adjust n_neighbors as needed
lle_reduced_data = lle.fit_transform(data_np)

visualization.plot(lle_reduced_data, labels, labels, np.empty(0), "./Results/r100/LLE")

### LapEig

In [ ]:
spectral_embedding = SpectralEmbedding(n_components=2, affinity='rbf')
spectral_reduced_data = spectral_embedding.fit_transform(data_np)

visualization.plot(spectral_reduced_data, labels, labels, np.empty(0), "./Results/r100/LapEig")

### Sae